In [1]:
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from geopy.geocoders import Nominatim
%matplotlib inline 

In [2]:
crashes_df = gpd.read_file('data/crashes_cleaned.geojson')

In [8]:
crashes_df.set_crs(crs='EPSG:4326', inplace=True)

,injuries_fatal,injuries_non_incapacitating,crash_record_id,injuries_incapacitating,injuries_no_indication,latitude,street_no,injuries_unknown,device_condition,crash_date,...,prim_contributory_cause,sec_contributory_cause,street_direction,posted_speed_limit,injuries_total,lane_cnt,full_address,no_significant_injury,injuries_not_significant,geometry
0,NaN,NaN,0edc78f89df5b72ddaa1c1f567e229ad39c0e098953a9d...,NaN,NaN,41.873520168,3357,NaN,NO CONTROLS,2022-08-09T01:40:00,...,FAILING TO REDUCE SPEED TO AVOID CRASH,NOT APPLICABLE,W,30,NaN,None,3357 W HARRISON ST CHICAGO IL,NaN,NaN,POINT (-87.71064 41.87352)
1,0.0,0.0,49ff3f04d16f5a71e6d66436c94f6acfd939c20d6c3651...,0.0,2.0,41.831835811,3410,0.0,FUNCTIONING PROPERLY,2022-08-09T00:45:00,...,FOLLOWING TOO CLOSELY,NOT APPLICABLE,S,30,0.0,None,3410 S DAMEN AVE CHICAGO IL,2.0,2.0,POINT (-87.67546 41.83184)
2,NaN,NaN,e580e89f187525bf685101a36fc64df499a72be926d5a9...,NaN,NaN,41.838371536,130,NaN,FUNCTIONING PROPERLY,2022-08-08T23:55:00,...,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,W,35,NaN,None,130 W 31ST ST CHICAGO IL,NaN,NaN,POINT (-87.63002 41.83837)
3,0.0,0.0,cdd7c5d90668e4d1bda12805ad19cec305667643a88806...,0.0,1.0,41.861481407,2747,0.0,NO CONTROLS,2022-08-08T23:00:00,...,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,30,0.0,None,2747 W OGDEN AVE CHICAGO IL,1.0,1.0,POINT (-87.69494 41.86148)
4,0.0,0.0,e7c26ad2dd7f250b14acafe2d86265616f59f7c642a505...,0.0,2.0,41.837964154,3100,0.0,FUNCTIONING PROPERLY,2022-08-08T23:00:00,...,DISREGARDING TRAFFIC SIGNALS,DISREGARDING TRAFFIC SIGNALS,S,35,0.0,None,3100 S HALSTED ST CHICAGO IL,2.0,2.0,POINT (-87.64642 41.83796)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631400,0.0,0.0,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,0.0,2.0,41.835886103,3132,0.0,NO CONTROLS,2014-06-25T19:00:00,...,IMPROPER OVERTAKING/PASSING,NOT APPLICABLE,S,30,0.0,None,3132 S PULASKI RD CHICAGO IL,2.0,2.0,POINT (-87.72447 41.83589)
631401,0.0,0.0,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,0.0,2.0,41.884016475,199,0.0,FUNCTIONING PROPERLY,2014-02-24T19:45:00,...,UNABLE TO DETERMINE,UNABLE TO DETERMINE,N,30,0.0,None,199 N SACRAMENTO BLVD CHICAGO IL,2.0,2.0,POINT (-87.70114 41.88402)
631402,0.0,1.0,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,0.0,1.0,41.760710194,7400,0.0,NO CONTROLS,2014-01-21T07:40:00,...,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,S,30,1.0,None,7400 S EXCHANGE AVE CHICAGO IL,1.0,1.0,POINT (-87.56195 41.76071)
631403,0.0,0.0,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,0.0,2.0,41.885609917,415,0.0,NO CONTROLS,2014-01-18T18:14:00,...,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,30,0.0,None,415 W LAKE ST CHICAGO IL,2.0,2.0,POINT (-87.63876 41.88561)


In [9]:
crashes_df.head()

,injuries_fatal,injuries_non_incapacitating,crash_record_id,injuries_incapacitating,injuries_no_indication,latitude,street_no,injuries_unknown,device_condition,crash_date,...,prim_contributory_cause,sec_contributory_cause,street_direction,posted_speed_limit,injuries_total,lane_cnt,full_address,no_significant_injury,injuries_not_significant,geometry
0,NaN,NaN,0edc78f89df5b72ddaa1c1f567e229ad39c0e098953a9d...,NaN,NaN,41.873520168,3357,NaN,NO CONTROLS,2022-08-09T01:40:00,...,FAILING TO REDUCE SPEED TO AVOID CRASH,NOT APPLICABLE,W,30,NaN,None,3357 W HARRISON ST CHICAGO IL,NaN,NaN,POINT (-87.71064 41.87352)
1,0.0,0.0,49ff3f04d16f5a71e6d66436c94f6acfd939c20d6c3651...,0.0,2.0,41.831835811,3410,0.0,FUNCTIONING PROPERLY,2022-08-09T00:45:00,...,FOLLOWING TOO CLOSELY,NOT APPLICABLE,S,30,0.0,None,3410 S DAMEN AVE CHICAGO IL,2.0,2.0,POINT (-87.67546 41.83184)
2,NaN,NaN,e580e89f187525bf685101a36fc64df499a72be926d5a9...,NaN,NaN,41.838371536,130,NaN,FUNCTIONING PROPERLY,2022-08-08T23:55:00,...,FAILING TO YIELD RIGHT-OF-WAY,NOT APPLICABLE,W,35,NaN,None,130 W 31ST ST CHICAGO IL,NaN,NaN,POINT (-87.63002 41.83837)
3,0.0,0.0,cdd7c5d90668e4d1bda12805ad19cec305667643a88806...,0.0,1.0,41.861481407,2747,0.0,NO CONTROLS,2022-08-08T23:00:00,...,UNABLE TO DETERMINE,UNABLE TO DETERMINE,W,30,0.0,None,2747 W OGDEN AVE CHICAGO IL,1.0,1.0,POINT (-87.69494 41.86148)
4,0.0,0.0,e7c26ad2dd7f250b14acafe2d86265616f59f7c642a505...,0.0,2.0,41.837964154,3100,0.0,FUNCTIONING PROPERLY,2022-08-08T23:00:00,...,DISREGARDING TRAFFIC SIGNALS,DISREGARDING TRAFFIC SIGNALS,S,35,0.0,None,3100 S HALSTED ST CHICAGO IL,2.0,2.0,POINT (-87.64642 41.83796)


In [4]:
peds_df = pd.read_csv('data/peds_cleaned_df.csv')

In [5]:
peds_df.head()

,Unnamed: 0,person_id,person_type,crash_record_id,crash_date,city,state,zipcode,sex,age,...,driver_action,driver_vision,physical_condition,pedpedal_action,pedpedal_visibility,pedpedal_location,bac_result,rd_no,cell_phone_use,bac_result_value
0,0,O1395114,PEDESTRIAN,10c02ad83b1b241f057db7e3f4bebdb9a9a2798dc57db4...,2022-08-08T17:22:00.000,CHAMPAIGN,IL,61820,F,21.0,...,NONE,UNKNOWN,NORMAL,CROSSING - WITH SIGNAL,NO CONTRASTING CLOTHING,IN CROSSWALK,TEST NOT OFFERED,NaN,NaN,NaN
1,1,O1395064,PEDESTRIAN,e287152209a3cf769764c1df04fa42bc3eddafd1df32ef...,2022-08-08T17:12:00.000,OAK PARK,IL,60302,M,33.0,...,UNKNOWN,UNKNOWN,NORMAL,CROSSING - WITH SIGNAL,NO CONTRASTING CLOTHING,IN CROSSWALK,TEST NOT OFFERED,NaN,NaN,NaN
2,2,O1395025,PEDESTRIAN,862babdabad4148c0e2345ae956733d6c0d3e4fb5b5f11...,2022-08-08T15:34:00.000,ARLINGTON HTS.,IL,60004,F,54.0,...,NONE,NOT OBSCURED,NORMAL,CROSSING - WITH SIGNAL,CONTRASTING CLOTHING,IN CROSSWALK,TEST NOT OFFERED,NaN,NaN,NaN
3,3,O1395061,PEDESTRIAN,1a12b6c50c25d0f049e4d33dce433272217e0d220f8963...,2022-08-08T14:54:00.000,CHICAGO,IL,60605,M,33.0,...,NaN,NaN,NaN,CROSSING - AGAINST SIGNAL,NO CONTRASTING CLOTHING,IN ROADWAY,TEST NOT OFFERED,NaN,NaN,NaN
4,4,O1394836,PEDESTRIAN,94d427ca74695d355d9dad4d717140f436aeb16f447a24...,2022-08-08T09:28:00.000,CHICAGO,IL,60619,F,25.0,...,OTHER,NOT OBSCURED,NORMAL,WITH TRAFFIC,NO CONTRASTING CLOTHING,DRIVEWAY ACCESS,TEST NOT OFFERED,NaN,NaN,NaN


In [6]:
cyclists_df = pd.read_csv('data/cyclists_cleaned_df.csv')

In [7]:
cyclists_df

,Unnamed: 0,person_id,person_type,crash_record_id,crash_date,sex,safety_equipment,ejection,injury_classification,driver_action,...,bac_result,city,state,hospital,zipcode,age,ems_agency,rd_no,bac_result_value,cell_phone_use
0,0,O1395052,BICYCLE,73c7c72402a0325ee3217b85b97f1cd880dce1e273294e...,2022-08-08T16:35:00.000,M,HELMET NOT USED,PARTIALLY EJECTED,NO SIGNIFICANT INJURY,DISREGARDED CONTROL DEVICES,...,TEST NOT OFFERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,O1394929,BICYCLE,e7b51330cb3cffbe562d80eb7d25cce413dcf4d3d6f8f2...,2022-08-08T14:12:00.000,NaN,HELMET NOT USED,UNKNOWN,NO SIGNIFICANT INJURY,WRONG WAY/SIDE,...,TEST NOT OFFERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,O1394880,BICYCLE,8fda3f8d995b2c9ba0cd7cbe35f60aacd6708defbd91db...,2022-08-08T13:00:00.000,M,HELMET NOT USED,NONE,NO SIGNIFICANT INJURY,IMPROPER TURN,...,TEST NOT OFFERED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,O1394456,BICYCLE,e5bb72ff7b983e5d06b7419143e5bf9a6f6028c84cb0de...,2022-08-07T19:35:00.000,M,HELMET NOT USED,NONE,NONINCAPACITATING INJURY,FAILED TO YIELD,...,TEST NOT OFFERED,CHICAGO,IL,SELF TRANSPORT,NaN,NaN,NaN,NaN,NaN,NaN
4,4,O1394398,BICYCLE,37a4a38e3879c073141bbe1259d755e6c9706c732e77c6...,2022-08-07T17:19:00.000,M,HELMET NOT USED,UNKNOWN,NONINCAPACITATING INJURY,NONE,...,TEST NOT OFFERED,CHICAGO,IL,ST. BERNARD'S,60621,19.0,CFD AMBO 24,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9470,9470,O676,BICYCLE,0ebcfc991b206474cf68ef0099609684dffb3b0c1e078e...,2015-08-26T08:41:00.000,M,HELMET NOT USED,TOTALLY EJECTED,NO SIGNIFICANT INJURY,NONE,...,TEST NOT OFFERED,CHICAGO,IL,NaN,60614,27.0,NaN,HY397895,NaN,NaN
9471,9471,O464,BICYCLE,a2458d8a19e7f502ec81f85ecdbd971c1e4bb12dc34da8...,2015-08-21T09:30:00.000,M,HELMET USED,PARTIALLY EJECTED,NONINCAPACITATING INJURY,NaN,...,TEST NOT OFFERED,CHICAGO,IL,NaN,60622,52.0,NaN,HY391578,NaN,NaN
9472,9472,O413,BICYCLE,3d25ae31f7adea360bd4b6fb138c541a84443dae0fad58...,2015-08-19T18:45:00.000,M,HELMET USED,NONE,NO SIGNIFICANT INJURY,OTHER,...,TEST NOT OFFERED,CHICAGO,IL,NaN,60605,31.0,NaN,HY389383,NaN,NaN
9473,9473,O967,BICYCLE,960536ce2853fdd8c904e8ed3ae0ba186535e0049a999b...,2015-08-19T09:15:00.000,M,HELMET USED,TOTALLY EJECTED,NO SIGNIFICANT INJURY,UNKNOWN,...,TEST NOT OFFERED,CHICAGO,IL,NaN,60647,33.0,NaN,HY406474,NaN,NaN
